In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [3]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[sql_query]
)

In [5]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [6]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='f7d3e7e6-2893-4fa7-84a8-3d89cb65176e'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 872, 'prompt_tokens': 142, 'total_tokens': 1014, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D93F6YpjnZreLYBqeNcKgzDf1O85R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5acb-22fe-7a81-ab5c-aef773bbdfad-0', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT name FROM artists WHERE name LIKE 'S%' ORDER BY popularity DESC LIMIT 1;"}, 'id': 'call_iGEidtkb1hPgiZcW8R54XOmU'

In [7]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT ar.Name AS ArtistName, SUM(il.UnitPrice * il.Quantity) AS Revenue FROM Artist ar JOIN Album al ON al.ArtistId = ar.ArtistId JOIN Track t ON t.AlbumId = al.AlbumId JOIN InvoiceLine il ON il.TrackId = t.TrackId WHERE ar.Name LIKE 'S%' GROUP BY ar.Name ORDER BY Revenue DESC LIMIT 1;
